# AKS Cookbook

## 🧪 Zero to JupyterHub with AKS

![visual](visual.png)

JupyterHub allows users to interact with a computing environment through a webpage. As most devices have access to a web browser, JupyterHub makes it is easy to provide and standardize the computing environment for a group of people (e.g., for a class of students or an analytics team).  
It's based on the official [Jupyter documentation](https://z2jh.jupyter.org/en/stable/).  

▶️ Click on the `Run All` button to execute all the subsequent steps in sequence, or run each step individually by executing the cells one at a time.

### TOC

- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Verify the Azure CLI and connected Azure subscription](#1)
- [2️⃣ Create a new Azure Resource Group or reuse an existing one](#2)
- [3️⃣ Create an AKS cluster](#3)
- [4️⃣ Connect to the AKS cluster](#4)
- [5️⃣ Install JupyterHub with Helm](#5)
- [6️⃣ Retrieve the ingress IP to access the JupyterHub](#6)
- [🗑️ Clean up resources](#clean)


<a id='0'></a>
### 0️⃣ Initialize notebook variables
Adjust the location parameters according your preferences and on the [product availability by Azure region](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable).

In [ ]:
import os, time, json, requests, utils, random
create_resources = True # specify True if you want to create new resources, False to use existing ones

if create_resources:
    # create new resources with the following properties
    deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
    resource_group_name = f"lab-{deployment_name}" # change the name to match your naming convention
    resource_group_location = "eastus2"
    aks_resource_name = "aks-cluster"
    aks_node_count = 1

    dns_label = f"jupyterhub-{random.randrange(0,100)}" 

else:
    # or use the following existing resources
    resource_group_name = ""
    aks_resource_name = ""

utils.print_ok('Notebook initiaized')


<a id='1'></a>
### 1️⃣ Verify the Azure CLI and connected Azure subscription
The following commands ensure that you have the latest version of the Azure CLI and relevant extensions installed while also verifying that the Azure CLI is connected to your Azure subscription.

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")
if output.success and output.json_data:
    current_user = output.json_data.get('user').get('name')
    subsciption_id = output.json_data.get('id')
    tenant_id = output.json_data.get('tenantId')
output = utils.run("az provider register --namespace Microsoft.ContainerService --wait", "Microsoft.ContainerService registered in your subscription", "Failed to register Microsoft.ContainerService")
output = utils.run("az provider register --namespace Microsoft.KubernetesConfiguration --wait", "Microsoft.KubernetesConfiguration registered in your subscription", "Failed to register Microsoft.KubernetesConfiguration")
output = utils.run("az extension add --name k8s-extension", "az k8s-extension installed", "Failed to install az k8s-extension")
output = utils.run("az extension update --name k8s-extension", "az k8s-extension updated", "Failed to update az k8s-extension")
output = utils.run("az extension add --name aks-preview", "az aks-preview extension installed", "Failed to install az aks-preview extension")
output = utils.run("az extension update --name aks-preview", "az aks-preview extension updated", "Failed to update az aks-preview extension")


<a id='2'></a>
### 2️⃣ Create a new Azure Resource Group or reuse an existing one
All resources deployed in this lab will be created within the designated resource group. 

In [ ]:
if create_resources:
    utils.create_resource_group(create_resources, resource_group_name, resource_group_location)


<a id='3'></a>
### 3️⃣ Create an AKS cluster
The following step creates an AKS cluster using the [az aks create](https://learn.microsoft.com/en-us/cli/azure/aks?view=azure-cli-latest#az-aks-create) command. 

In [ ]:
if create_resources:
    output = utils.run(f"az aks create --resource-group {resource_group_name} --name {aks_resource_name} --node-count {aks_node_count} --generate-ssh-keys",
             f"AKS cluster '{aks_resource_name}' created",
             f"Failed to create AKS cluster '{aks_resource_name}'")




<a id='4'></a>
### 4️⃣ Connect to the AKS cluster
Configure kubectl to connect to your Kubernetes cluster using the [az aks get-credentials](https://learn.microsoft.com/en-us/cli/azure/aks?view=azure-cli-latest#az-aks-get-credentials) command. This command downloads credentials and configures the Kubernetes CLI to use them.

In [ ]:
output = utils.run(f"az aks get-credentials --resource-group {resource_group_name} --name {aks_resource_name} --overwrite-existing",
             f"Credentials for AKS cluster '{aks_resource_name}' configured",
             f"Failed to configure credentials for AKS cluster '{aks_resource_name}'")


output = utils.run(f"az aks show --resource-group {resource_group_name} --name {aks_resource_name} --only-show-errors",
            f"AKS cluster '{aks_resource_name}' retrieved",
            f"Failed to retrieve AKS cluster '{aks_resource_name}'")
if output.success and output.json_data:
    aks_node_resource_group = output.json_data.get('nodeResourceGroup')



<a id='5'></a>
### 5️⃣ Install JupyterHub with Helm



In [ ]:
! helm repo add jupyterhub https://hub.jupyter.org/helm-chart/
! helm repo update

! helm upgrade --cleanup-on-fail --install "jupyterhub" jupyterhub/jupyterhub --namespace "jupyterhub" --create-namespace --version=3.3.8 --values config.yaml

<a id='6'></a>
### 6️⃣ Retrieve the ingress IP to access the JupyterHub

In [ ]:
output = utils.run("kubectl get service proxy-public -n jupyterhub -o json", "Retrieved proxy-public service", "Failed to get proxy-public service")
if output.success and output.json_data:
    proxy_public_ip = output.json_data.get('status').get('loadBalancer').get('ingress')[0].get('ip')

    output = utils.run(f"az network public-ip list --resource-group {aks_node_resource_group} --query \"[?ipAddress=='{proxy_public_ip}']\"", "Retrieved public ip", "Failed to retrieve public ip") 
    if output.success and output.json_data:
        ip_id = output.json_data[0]['id']

        utils.run(f"az network public-ip update --ids {ip_id} --dns-name {dns_label}", "Updated public ip with dns label", "Failed to update public ip with dns label")
        if output.success and output.json_data:
            fqdn = output.json_data[0].get('dnsSettings').get('fqdn')
            utils.print_ok(f"JupyterHub is available at http://{proxy_public_ip}")
            utils.print_ok(f"JupyterHub is available at http://{fqdn}")
            utils.print_info(f"You can enter any username and password to login")



<a id='clean'></a>
### 🗑️ Clean up resources
When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered. Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.